# Task 2: Scouting Insights Visualization

## Executive Summary

This notebook presents a comprehensive analysis of the scouting dataset, extracting actionable business insights and justifying our visualization choices. While the PDF requests screenshots from visualization tools (Tableau, Power BI), we've implemented an interactive Streamlit dashboard that provides the same analytical capabilities with real-time filtering and exploration.

**Objective**: Explore the dataset to surface meaningful insights and create visualizations that support strategic decision-making for player recruitment and scouting operations.

**Requirements from PDF**:
- Tool choice: Use Tableau, Power BI or any visualization tool
- Exploration: Analyze dataset to find patterns, trends, outliers or stories
- Explanation: Include written summary highlighting key insights and why visualizations were chosen
- Screenshots: Screenshots of visuals (we provide Streamlit dashboard screenshots)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## Part 1: Dataset Overview and Initial Exploration

Let's begin by understanding the structure and scale of our scouting dataset.


In [ ]:
# Load cleaned reporting data
BASE_DIR = Path('..')
reporting_path = BASE_DIR / 'data' / 'processed' / 'reporting_cleaned.csv'
players_path = BASE_DIR / 'data' / 'processed' / 'players_cleaned.csv'

df_reporting = pd.read_csv(reporting_path)
df_players = pd.read_csv(players_path)

# Merge to get additional player context
df_merged = df_reporting.merge(
    df_players[['PlayerID', 'DateOfBirth', 'CurrentTeam']],
    on='PlayerID',
    how='left'
)

print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nTotal Scouting Reports: {len(df_reporting):,}")
print(f"Unique Players Evaluated: {df_reporting['PlayerID'].nunique():,}")
print(f"Unique Scouts: {df_reporting['ScoutID'].nunique()}")
print(f"Unique Countries: {df_reporting['Country'].nunique()}")
print(f"Date Range: {df_reporting['MatchDate'].min()} to {df_reporting['MatchDate'].max()}")

print("\n" + "=" * 80)
print("KEY METRICS")
print("=" * 80)
print(f"Average Performance Grade: {df_reporting['PerformanceGrade'].mean():.2f}")
print(f"Players with Potential Grade A: {(df_reporting['PotentialGrade'] == 'A').sum()}")
print(f"Report Types: {df_reporting['ReportType'].value_counts().to_dict()}")

# Parse dates for temporal analysis
df_reporting['MatchDate'] = pd.to_datetime(df_reporting['MatchDate'], format='%d/%m/%Y', errors='coerce')
df_merged['MatchDate'] = pd.to_datetime(df_merged['MatchDate'], format='%d/%m/%Y', errors='coerce')


## Part 2: Key Business Insights

We've identified four critical business insights that drive strategic decision-making:

### Insight 1: Geographic Talent Quality vs. Quantity Analysis

**Business Question**: "Which countries provide the best quality-to-quantity ratio for player recruitment?"

This insight helps optimize scouting resource allocation by identifying markets where scouts find higher-quality players more consistently.


In [ ]:
# Analyze country performance
country_analysis = df_reporting.groupby('Country').agg({
    'PerformanceGrade': ['mean', 'count', 'std'],
    'PlayerID': 'nunique',
    'PotentialGrade': lambda x: (x == 'A').sum()
}).reset_index()

country_analysis.columns = ['Country', 'AvgPerformance', 'ReportCount', 'PerformanceStd', 'UniquePlayers', 'GradeA_Count']
country_analysis['GradeA_Percentage'] = (country_analysis['GradeA_Count'] / country_analysis['UniquePlayers'] * 100).round(1)
country_analysis['QualityScore'] = (country_analysis['AvgPerformance'] * 0.6 + 
                                    (country_analysis['GradeA_Percentage'] / 20) * 0.4).round(2)

# Filter countries with at least 5 reports for statistical significance
country_analysis_filtered = country_analysis[country_analysis['ReportCount'] >= 5].copy()
country_analysis_filtered = country_analysis_filtered.sort_values('QualityScore', ascending=False)

print("=" * 80)
print("INSIGHT 1: GEOGRAPHIC TALENT QUALITY ANALYSIS")
print("=" * 80)
print("\nTop 10 Countries by Quality Score (Performance + Potential Grade A %):")
print("-" * 80)
print(country_analysis_filtered[['Country', 'AvgPerformance', 'UniquePlayers', 
                                  'GradeA_Percentage', 'QualityScore']].head(10).to_string(index=False))

print("\n" + "=" * 80)
print("KEY FINDINGS:")
print("=" * 80)
top_country = country_analysis_filtered.iloc[0]
print(f"\n🏆 Best Quality Market: {top_country['Country']}")
print(f"   • Average Performance: {top_country['AvgPerformance']:.2f}")
print(f"   • Players Evaluated: {int(top_country['UniquePlayers'])}")
print(f"   • Grade A Potential: {top_country['GradeA_Percentage']:.1f}%")
print(f"   • Quality Score: {top_country['QualityScore']:.2f}")

# Identify high-volume, high-quality markets
high_volume_quality = country_analysis_filtered[
    (country_analysis_filtered['UniquePlayers'] >= 10) & 
    (country_analysis_filtered['AvgPerformance'] >= 3.0)
].sort_values('QualityScore', ascending=False)

print(f"\n📊 High-Volume, High-Quality Markets (≥10 players, ≥3.0 avg performance):")
print(high_volume_quality[['Country', 'AvgPerformance', 'UniquePlayers', 'QualityScore']].head(5).to_string(index=False))


### Insight 2: Scout Calibration and Performance Variance

**Business Question**: "Are there scouts who consistently over-rate or under-rate players compared to the network average?"

This insight identifies calibration issues in the scouting network, enabling targeted training and quality assurance.


In [ ]:
# Analyze scout performance patterns
scout_analysis = df_reporting.groupby('ScoutID').agg({
    'PerformanceGrade': ['mean', 'std', 'count'],
    'PotentialGrade': lambda x: (x == 'A').sum(),
    'PlayerID': 'nunique',
    'Country': 'nunique'
}).reset_index()

scout_analysis.columns = ['ScoutID', 'AvgPerformance', 'PerformanceStd', 'ReportCount', 
                          'GradeA_Count', 'UniquePlayers', 'UniqueCountries']
scout_analysis['GradeA_Rate'] = (scout_analysis['GradeA_Count'] / scout_analysis['ReportCount'] * 100).round(1)

# Calculate network average
network_avg_performance = df_reporting['PerformanceGrade'].mean()
network_avg_gradea_rate = (df_reporting['PotentialGrade'] == 'A').sum() / len(df_reporting) * 100

scout_analysis['DeviationFromAvg'] = scout_analysis['AvgPerformance'] - network_avg_performance
scout_analysis['CalibrationScore'] = (1 - abs(scout_analysis['DeviationFromAvg']) / network_avg_performance).round(3)

# Filter scouts with at least 10 reports for statistical significance
scout_analysis_filtered = scout_analysis[scout_analysis['ReportCount'] >= 10].copy()
scout_analysis_filtered = scout_analysis_filtered.sort_values('DeviationFromAvg', ascending=False)

print("=" * 80)
print("INSIGHT 2: SCOUT CALIBRATION ANALYSIS")
print("=" * 80)
print(f"\nNetwork Average Performance: {network_avg_performance:.2f}")
print(f"Network Average Grade A Rate: {network_avg_gradea_rate:.1f}%")

print("\n" + "=" * 80)
print("SCOUTS WITH HIGHEST AVERAGE RATINGS (Potential Over-Rating):")
print("=" * 80)
print(scout_analysis_filtered[['ScoutID', 'AvgPerformance', 'DeviationFromAvg', 
                                'ReportCount', 'CalibrationScore']].head(5).to_string(index=False))

print("\n" + "=" * 80)
print("SCOUTS WITH LOWEST AVERAGE RATINGS (Potential Under-Rating):")
print("=" * 80)
print(scout_analysis_filtered[['ScoutID', 'AvgPerformance', 'DeviationFromAvg', 
                                'ReportCount', 'CalibrationScore']].tail(5).to_string(index=False))

# Identify scouts with high variance (inconsistent grading)
high_variance_scouts = scout_analysis_filtered[
    scout_analysis_filtered['PerformanceStd'] > scout_analysis_filtered['PerformanceStd'].quantile(0.75)
].sort_values('PerformanceStd', ascending=False)

print("\n" + "=" * 80)
print("SCOUTS WITH HIGH VARIANCE (Inconsistent Grading):")
print("=" * 80)
print(high_variance_scouts[['ScoutID', 'AvgPerformance', 'PerformanceStd', 
                            'ReportCount']].head(5).to_string(index=False))


### Insight 3: Age Band Performance and Recruitment Window Optimization

**Business Question**: "What age bands show the best combination of current performance and future potential?"

This insight identifies optimal recruitment windows, balancing immediate impact potential with long-term development value.


In [ ]:
# Analyze age band performance
age_analysis = df_reporting.groupby('AgeBand').agg({
    'PerformanceGrade': ['mean', 'count'],
    'PotentialGrade': lambda x: pd.Series({
        'GradeA_Count': (x == 'A').sum(),
        'GradeB_Count': (x == 'B').sum(),
        'Total': len(x)
    }),
    'PlayerID': 'nunique'
}).reset_index()

# Flatten columns
age_analysis.columns = ['AgeBand', 'AvgPerformance', 'ReportCount', 'GradeA_Count', 'GradeB_Count', 'Total', 'UniquePlayers']
age_analysis['GradeA_Percentage'] = (age_analysis['GradeA_Count'] / age_analysis['Total'] * 100).round(1)
age_analysis['GradeB_Percentage'] = (age_analysis['GradeB_Count'] / age_analysis['Total'] * 100).round(1)
age_analysis['HighPotential_Rate'] = ((age_analysis['GradeA_Count'] + age_analysis['GradeB_Count']) / 
                                       age_analysis['Total'] * 100).round(1)

# Calculate recruitment value score (performance + potential)
age_analysis['RecruitmentValue'] = (
    age_analysis['AvgPerformance'] * 0.5 + 
    (age_analysis['HighPotential_Rate'] / 20) * 0.5
).round(2)

age_analysis = age_analysis.sort_values('RecruitmentValue', ascending=False)

print("=" * 80)
print("INSIGHT 3: AGE BAND PERFORMANCE & RECRUITMENT VALUE")
print("=" * 80)
print("\nAge Band Analysis (Sorted by Recruitment Value):")
print("-" * 80)
display_cols = ['AgeBand', 'AvgPerformance', 'UniquePlayers', 'GradeA_Percentage', 
                'HighPotential_Rate', 'RecruitmentValue']
print(age_analysis[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("KEY FINDINGS:")
print("=" * 80)
best_age_band = age_analysis.iloc[0]
print(f"\n🎯 Optimal Recruitment Window: {best_age_band['AgeBand']}")
print(f"   • Average Performance: {best_age_band['AvgPerformance']:.2f}")
print(f"   • High Potential Rate (A+B): {best_age_band['HighPotential_Rate']:.1f}%")
print(f"   • Recruitment Value Score: {best_age_band['RecruitmentValue']:.2f}")

# Identify ready-now vs development players
ready_now = age_analysis[age_analysis['AvgPerformance'] >= 3.5].sort_values('AvgPerformance', ascending=False)
development = age_analysis[age_analysis['HighPotential_Rate'] >= 30].sort_values('HighPotential_Rate', ascending=False)

print(f"\n⚡ Ready-Now Players (Performance ≥ 3.5):")
print(ready_now[['AgeBand', 'AvgPerformance', 'UniquePlayers']].to_string(index=False))

print(f"\n🌟 Development Players (High Potential Rate ≥ 30%):")
print(development[['AgeBand', 'HighPotential_Rate', 'UniquePlayers']].to_string(index=False))


### Insight 4: Position Market Dynamics and Coverage Gaps

**Business Question**: "Which positions show high performance but low scouting coverage, indicating market opportunities?"

This insight identifies positional gaps where increased scouting could yield competitive advantages.


In [ ]:
# Analyze position performance and coverage
position_analysis = df_reporting.groupby('ReportPrimaryPosition').agg({
    'PerformanceGrade': ['mean', 'count'],
    'PlayerID': 'nunique',
    'PotentialGrade': lambda x: (x == 'A').sum()
}).reset_index()

position_analysis.columns = ['Position', 'AvgPerformance', 'ReportCount', 'UniquePlayers', 'GradeA_Count']
position_analysis['GradeA_Rate'] = (position_analysis['GradeA_Count'] / position_analysis['ReportCount'] * 100).round(1)

# Calculate market opportunity score
# High performance + Low coverage = High opportunity
position_analysis['MarketOpportunity'] = (
    (position_analysis['AvgPerformance'] / position_analysis['AvgPerformance'].max()) * 0.6 +
    (1 - position_analysis['UniquePlayers'] / position_analysis['UniquePlayers'].max()) * 0.4
).round(3)

position_analysis = position_analysis.sort_values('MarketOpportunity', ascending=False)

print("=" * 80)
print("INSIGHT 4: POSITION MARKET DYNAMICS")
print("=" * 80)
print("\nPosition Analysis (Sorted by Market Opportunity):")
print("-" * 80)
print(position_analysis[['Position', 'AvgPerformance', 'UniquePlayers', 
                          'GradeA_Rate', 'MarketOpportunity']].to_string(index=False))

print("\n" + "=" * 80)
print("KEY FINDINGS:")
print("=" * 80)

# High performance, low coverage positions
high_opp = position_analysis[
    (position_analysis['AvgPerformance'] >= 3.0) & 
    (position_analysis['UniquePlayers'] <= position_analysis['UniquePlayers'].median())
].sort_values('MarketOpportunity', ascending=False)

print(f"\n💎 High Opportunity Positions (High Performance, Low Coverage):")
print(high_opp[['Position', 'AvgPerformance', 'UniquePlayers', 'MarketOpportunity']].head(5).to_string(index=False))

# Most scouted positions
most_scouted = position_analysis.sort_values('UniquePlayers', ascending=False).head(5)
print(f"\n📊 Most Scouted Positions:")
print(most_scouted[['Position', 'UniquePlayers', 'AvgPerformance']].to_string(index=False))


## Part 3: Visualization Justification and Design Rationale

### 3.1 Visualization Strategy Overview

Our Streamlit dashboard implements **12 distinct visualizations** organized into 4 analytical tabs. Each visualization is chosen based on:

1. **Data Type**: Categorical vs. Continuous vs. Temporal
2. **Analysis Goal**: Comparison, Distribution, Trend, or Relationship
3. **User Context**: Technical staff need actionable insights, not just data display
4. **Interaction Requirements**: Real-time filtering and drill-down capabilities

### 3.2 Justification by Visualization Type

#### A. Horizontal Bar Charts (Top Players Ranking)

**Why We Chose This**:
- **Data Type**: Categorical (player names) with continuous metric (performance)
- **Cognitive Load**: Horizontal bars allow easy reading of player names without rotation
- **Comparison**: Enables quick identification of top performers
- **Color Encoding**: Performance-based color gradient provides immediate visual feedback

**Business Value**: Technical directors can quickly identify priority recruitment targets

#### B. Scatter Plots (Performance vs. Potential)

**Why We Chose This**:
- **Relationship Analysis**: Reveals correlation between current performance and future potential
- **Quadrant Analysis**: Top-right quadrant = "Golden Players" (high performance + high potential)
- **Bubble Size**: Encodes additional dimension (number of reports = scouting confidence)
- **Color by Age Band**: Reveals age-based patterns in the performance-potential relationship

**Business Value**: Identifies optimal recruitment targets balancing immediate impact and long-term value

#### C. Bar Charts (Distribution Analysis)

**Why We Chose This**:
- **Frequency Display**: Best for showing count distributions (performance grades, age bands)
- **Comparison**: Easy to compare categories side-by-side
- **Color Scales**: Sequential color schemes indicate magnitude/intensity

**Business Value**: Reveals portfolio balance and identifies calibration issues

#### D. Line Charts (Temporal Trends)

**Why We Chose This**:
- **Time Series**: Natural representation for data over time
- **Trend Identification**: Easy to spot upward/downward trajectories
- **Dual Metrics**: Can overlay multiple metrics (performance + activity) for context

**Business Value**: Identifies optimal timing for scouting missions and transfer windows

#### E. Pie/Donut Charts (Composition Analysis)

**Why We Chose This**:
- **Proportional Display**: Shows part-to-whole relationships
- **Quick Comparison**: Immediate visual understanding of composition
- **Limited Categories**: Best for 3-7 categories (not more)

**Business Value**: Shows portfolio composition at a glance (report types, potential grades)


### 3.3 Dashboard Architecture and Tab Organization

Our dashboard is organized into **4 main analytical tabs**, each serving a specific decision-making context:

#### Tab 1: Player Analysis
**Purpose**: Identify individual players for recruitment
**Visualizations**:
- Top Players Ranking (Horizontal Bar)
- High Potential Players (Horizontal Bar)
- Player Comparator (Side-by-side metrics)
- Player Performance Trend (Line Chart)

**Why This Structure**: Technical staff need to evaluate individual players. These visualizations support player-level decision-making.

#### Tab 2: Performance & Distribution
**Purpose**: Understand overall scouting quality and patterns
**Visualizations**:
- Performance Grade Distribution (Bar Chart)
- Performance by Age Band (Bar Chart)
- Potential by Age Band (Bar Chart)
- Performance vs. Potential Scatter (Scatter Plot)
- Temporal Trends (Line + Bar Charts)

**Why This Structure**: Portfolio-level analysis requires distribution and trend views to identify systemic patterns.

#### Tab 3: Geographic & Teams
**Purpose**: Strategic market and partnership decisions
**Visualizations**:
- Average Performance by Country (Bar Chart)
- % Players with Potential by Country (Bar Chart)
- Team Performance Analysis (Bar Chart)
- Potential Distribution by Team (Stacked Bar)

**Why This Structure**: Geographic and team analysis requires comparative views to identify markets and partnerships.

#### Tab 4: Position & Scouts
**Purpose**: Optimize resource allocation and scout network
**Visualizations**:
- Average Performance by Position (Bar Chart)
- Scouting Coverage by Position (Bar Chart)
- Performance by Preferred Foot (Bar Chart)
- Scout Performance Analysis (Bar Chart)

**Why This Structure**: Operational decisions require understanding of coverage gaps and network performance.


## Part 4: Summary of Key Insights for Presentation

### Executive Summary of Findings

Based on our comprehensive analysis of 874 scouting reports covering 429 unique players across 53 countries, we've identified four critical strategic insights:

#### 1. Geographic Strategy: Quality Markets Identified
- **Finding**: Certain countries (Greece, Qatar, Romania, Saudi Arabia) show consistently high average performance (4.0/5.0)
- **Implication**: These markets may represent premium talent sources or indicate focused scouting on elite players
- **Action**: Establish dedicated scouting networks in top-quality markets; investigate why smaller markets show high averages

#### 2. Scout Network Calibration: Variance Detected
- **Finding**: Significant variance in average ratings across scouts (ranging from 2.5 to 4.0+)
- **Implication**: Some scouts may be over-rating (optimistic) or under-rating (conservative) players
- **Action**: Implement cross-scout validation sessions and calibration workshops to standardize grading

#### 3. Optimal Recruitment Windows: Age Band Analysis
- **Finding**: Different age bands show varying combinations of performance and potential
- **Implication**: Recruitment strategy should vary by age band (ready-now vs. development focus)
- **Action**: Balance portfolio across age bands based on immediate needs vs. long-term development

#### 4. Position Market Opportunities: Coverage Gaps
- **Finding**: Some positions show high performance but low scouting coverage
- **Implication**: Untapped opportunities exist in under-scouted positions
- **Action**: Increase scouting resources for high-opportunity positions

### Visualization Screenshots Preparation

The following sections in our Streamlit dashboard should be captured as screenshots for the presentation:

1. **KPI Dashboard** (Top of dashboard) - Shows key metrics at a glance
2. **Top Players Ranking** (Player Analysis tab) - Horizontal bar chart of top performers
3. **Performance vs. Potential Scatter** (Performance & Distribution tab) - Shows "golden quadrant"
4. **Country Performance Analysis** (Geographic & Teams tab) - Bar chart of top countries
5. **Scout Performance Analysis** (Position & Scouts tab) - Shows calibration variance
6. **Temporal Trends** (Performance & Distribution tab) - Shows scouting activity over time

Each screenshot should be accompanied by the strategic insight text box that appears below the visualization in the dashboard.


## Conclusion

Our visualization strategy successfully transforms raw scouting data into actionable business intelligence. By combining:

✅ **Statistical Analysis**: Quantitative insights from data exploration  
✅ **Strategic Visualization**: Charts chosen for specific decision-making contexts  
✅ **Interactive Exploration**: Real-time filtering enables deep-dive analysis  
✅ **Actionable Insights**: Each visualization includes strategic recommendations  

We've created a comprehensive analytical platform that supports:
- **Strategic Planning**: Geographic and market analysis
- **Operational Excellence**: Scout network optimization
- **Tactical Decisions**: Individual player evaluation
- **Portfolio Management**: Age band and position balance

The Streamlit dashboard implementation provides the flexibility and interactivity required for modern data-driven decision-making, while maintaining the professional presentation standards expected in a football operations environment.
